# 1. 数据准备

图像数据准备方法：

1、用tf.keras中的ImageDataGenerator工具构建图片数据生成器；

2、使用tf.data.Dataset搭配tf.image中的一些图片处理方法构建数据管道（TensorFlow的原生方法）。

本文使用第2中方法准备图像数据

图像说明：

cifar2数据集为cifar10数据集的子集，只包括前两种类别airplane和automobile。

训练集有airplane和automobile图片各5000张，测试集有airplane和automobile图片各1000张。

cifar2任务的目标是训练一个模型来对飞机airplane和机动车automobile两种图片进行分类

In [37]:
import os
import datetime
import tensorflow as tf
from tensorflow.keras import models, layers, datasets
from tensorboard import notebook
import pandas as pd
import matplotlib.pyplot as plt

BATCH_SIZE = 100

In [38]:
def load_image(image_path, size=(32, 32)):
    # 正则匹配image_path与“.*automobile.*”
    if tf.strings.regex_full_match(image_path, '.*automobile.*'):
        label = tf.constant(1, tf.int8)
    else:
        label = tf.constant(0, tf.int8)
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, size) / 255
    return img, label

In [39]:
# 使用并行化预处理num_parallel_calls 和预存数据prefetch来提升性能
ds_train = tf.data.Dataset.list_files('../data/cifar2/train/*/*.jpg') \
            .map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE) \
            .shuffle(buffer_size=1000).batch(BATCH_SIZE) \
            .prefetch(tf.data.experimental.AUTOTUNE)
ds_test = tf.data.Dataset.list_files('../data/cifar2/test/*/*.jpg') \
            .map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE) \
            .batch(BATCH_SIZE) \
            .prefetch(tf.data.experimental.AUTOTUNE)

In [36]:
# 查看部分数据
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.figure(figsize=(8, 8))
for i, (img, label) in enumerate(ds_train.unbatch.take(8)):
    ax = plt.subplot(4,2,i+1)
    ax.imshow(img.numpy())
    ax.set_title('')

(100, 32, 32, 3)
